<a href="https://colab.research.google.com/github/truptiwagh13/CL-PR-Codes/blob/main/CL_2_PR_part_of_speech_tagging_using_HMM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Importing the libraries
!pip install pomegranate
import numpy as np
import re
import pomegranate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 21.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pomegranate: filename=pomegranate-0.14.8-cp39-cp39-linux_x86_64.whl size=20651757 sha256=54af2ffa94eb304a45c06cc364b1bba13b5af5027bb03288f9f930035de6fde0
  Stored in directory: /root/.cache/pip/wheels/32/d5/06/60c731fe8251f1ac8b79205e3f8009c551813b06805bb35370
Successfully built pomegranate


In [36]:
#Initialize the Hidden Markov Model (HMM)
model=pomegranate.HiddenMarkovModel()

In [37]:
###Emission probabilities
#"Noun" emission and state
emission_noun=pomegranate.DiscreteDistribution({"mary":4/9,
                                                "jane":2/9,
                                                "will":1/9,
                                                "spot":2/9,
                                                "can":0,
                                                "see":0,
                                                "pat":0})
state_noun=pomegranate.State(emission_noun,name="noun")

In [38]:
emission_model=pomegranate.DiscreteDistribution({"mary":0,
                                                "jane":0,
                                                "will":3/4,
                                                "spot":0,
                                                "can":1/4,
                                                "see":0,
                                                "pat":0})
state_model=pomegranate.State(emission_noun,name="model")

In [39]:
emission_verb=pomegranate.DiscreteDistribution({"mary":0,
                                                "jane":0,
                                                "will":0,
                                                "spot":1/4,
                                                "can":0,
                                                "see":1/2,
                                                "pat":1/4})
state_verb=pomegranate.State(emission_noun,name="verb")

In [40]:
#Add the states into our HMM model
model.add_states(state_noun,state_model,state_verb)

In [42]:
###Transition probabilities

#Transition probabilities from "<s>" to "noun" ,"modal","verb"
model.add_transition(model.start,state_noun,3/4)
model.add_transition(model.start,state_model,1/4)
model.add_transition(model.start,state_verb,0)

In [44]:
###Transition probabilities

#Transition probabilities from "<s>" to "noun" ,"modal","verb"
model.add_transition(state_noun,state_noun,1/9)
model.add_transition(state_noun,state_model,1/3)
model.add_transition(state_noun,state_verb,1/9)

In [45]:
model.add_transition(state_model,state_noun,1)
model.add_transition(state_model,state_model,0)
model.add_transition(state_model,state_verb,3/4)

In [46]:
model.add_transition(state_verb,state_noun,1)
model.add_transition(state_verb,state_model,0)
model.add_transition(state_verb,state_verb,0)

In [47]:
model.add_transition(state_noun,model.end,4/9)
model.add_transition(state_model,model.end,0)
model.add_transition(state_verb,model.end,0)

In [48]:
model.bake()

In [49]:
print("Total number of edges : ",model.edge_count())
print("total number of states or nodes : ",model.node_count())


Total number of edges :  14
total number of states or nodes :  5


In [50]:
model.states[-1].name="<E>"


In [51]:
model.states[-2].name="<S>"

In [52]:
text="Jane will spot Will."

In [53]:
print(text)

Jane will spot Will.


In [54]:
text=text.lower()

In [55]:
text=re.sub(r"[^a-zA-Z0-9]"," ",text)

In [56]:
text=text.split()

In [57]:
print("preprocessed text :\n",text)

preprocessed text :
 ['jane', 'will', 'spot', 'will']


In [61]:
viterbi_likelihood,viterbi_path=model.viterbi(text)
viterbi_path=[i_state[1].name for i_state in viterbi_path]
print("The most likely sequence :\n",viterbi_path,"\n")
print("preprocessed text:\n",text)

The most likely sequence :
 ['<S>', 'noun', 'model', 'verb', 'noun', '<E>'] 

preprocessed text:
 ['jane', 'will', 'spot', 'will']
